<a href="https://www.kaggle.com/code/faiyazsadi/dominating-query-implementation?scriptVersionId=161729395" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%%writefile main.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

// using a char array as bit field
unsigned char array[125];
unsigned char array2[125];

int main()
{
    // read bit no b
    int b = 2;
    int value = (array[b/8] & (1 << (b % 8)));
    printf("get: %d\n", value);

    // set bit no b
    array[b/8] |= (1 << (b%8));
    value = (array[b/8] & (1 << (b % 8))) != 0;
    printf("set: %d\n", value);

    printf("compare before: %d\n", memcmp(array, array2, sizeof(array)));

    //clear bit no b
    array[b/8] &= ~(1 << (b%8));
    value = (array[b/8] & (1 << (b % 8))) != 0;
    printf("reset: %d\n", value);

    printf("compare after: %d\n", memcmp(array, array2, sizeof(array)));
    
    
    float f = 1.0;
    float result = ldexp(f, 16);
    result += 10;
    printf("float result is: %f\n", result);
    return 0;
}


Writing main.c


In [2]:
!gcc main.c -o main
!./main

get: 0
set: 1
compare before: 4
reset: 0
compare after: 0
float result is: 65546.000000


In [3]:
%%writefile dominating_query.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <math.h>

#define MAX_LINE_LENGTH     1024
#define MAX_FIELD_LENGTH    128
#define MAX_MOVIE_ID        1001
#define MAX_USER_ID         2001
#define NUM_OF_BUCKETS      11
#define TOPK                3


int ratings[MAX_MOVIE_ID][MAX_USER_ID];
unsigned int bucket[NUM_OF_BUCKETS][3000];
unsigned int result[NUM_OF_BUCKETS * TOPK];
//float result[NUM_OF_BUCKETS * TOPK];
int cur_idx[MAX_MOVIE_ID];
bool user[MAX_USER_ID];


int unique_user;

const double EPSILON = 1e-9;
int compareWeightedScore(const void *a, const void *b) {
    const float *record1 = (const float *) a;
    const float *record2 = (const float *) b;

    const float score1 = ldexp(*record1, -16);
    const float score2 = ldexp(*record2, -16);

    if(fabs(score1 - score2) < EPSILON) {
        return 0;
    } else if(score1 > score2) {
        return -1;
    } else {
        return 1;
    }
}

int compareScore(const void *a, const void *b) {
    const unsigned int *num1 = (const unsigned int *)a;
    const unsigned int *num2 = (const unsigned int *)b;
    if (*num1 > *num2) {
        return -1; // num1 comes before num2
    } else if (*num1 < *num2) {
        return 1;  // num2 comes before num1
    } else {
        return 0;  // Both numbers are equal
    }
}

int main()
{
    /* final.csv is sorted by movieId and then by userId. For each group of movieId the first 20 rows are picked. */
    FILE* file = fopen("final.csv", "r");
    if (file == NULL) {
        printf("Failed to open the file.\n");
        return 0;
    }
    memset(ratings, -1, sizeof(ratings));
//    for(int i = 0; i < MAX_MOVIE_ID; ++i) {
//        for(int j = 0; j < MAX_USER_ID; ++j) {
//            ratings[i][j] = -1.00;
//        }
//    }
    char line[MAX_LINE_LENGTH];
    int id_no = -1;
    while (fgets(line, sizeof(line), file)) {
        id_no++;
        if(id_no == 0) continue; // the first row has the names of the columns. omit that.
        int userId, movieId;
        int rating;
        sscanf(line, "%d,%d,%d", &userId, &movieId, &rating);
        if(movieId >= MAX_MOVIE_ID || userId >= MAX_USER_ID) continue;
        if(user[userId] == false) {
            user[userId] = true;
            unique_user += 1;
        }
//        printf("%d %d %d\n", movieId, userId, rating);
        ratings[movieId][userId] = rating;
    }
    fclose(file);

//    for(int i = 0; i < 10; ++i) {
//        for(int j = 0; j < 10; ++j) {
//            printf("MovieId: %d, UserId: %d, Rating: %d\n", i, j, ratings[i][j]);
//        }
//    }


    memset(cur_idx, 0, sizeof(cur_idx));

    for(unsigned int movie_id = 1; movie_id < MAX_MOVIE_ID; ++movie_id) {
        unsigned int sum = 0;
        for(int user_id = 1; user_id < MAX_USER_ID; ++user_id) {
            if(ratings[movie_id][user_id] == -1) continue;
            sum += ratings[movie_id][user_id];
        }
        printf("sum : %d\n", sum);
        unsigned int score = movie_id;
        score |= (sum << 16);
//        char buffer[33];
//        itoa(score, buffer, 2);
//        printf("binary: %s\n", buffer);
        unsigned int bucket_id = movie_id % NUM_OF_BUCKETS;
        bucket[bucket_id][cur_idx[bucket_id]] = score;
        cur_idx[bucket_id]++;
        printf("movie_id: %d cur_idx: %d\n", movie_id, cur_idx[bucket_id]);
    }
    for(int i = 0; i < 10; ++i) {
        printf("%d\n", (bucket[1][i] << 16) >> 16);
    }
    for(int i = 0; i < NUM_OF_BUCKETS; ++i) {
        printf("%d, ", cur_idx[i]);
        qsort(bucket[i], (size_t) cur_idx[i], sizeof(unsigned int), compareScore);
    }
    printf("\n");
//    for(int i = 0; i < cur_idx[0]; ++i) {
//        printf("%d\n", bucket[0][i]);
//    }
    int at = 0;
    for(int i = 0; i < NUM_OF_BUCKETS; ++i) {
        for(int top = 0; top < TOPK; ++top) {
            result[at] = bucket[i][top];
            at++;
        }
    }

    int TKD[at];
    memset(TKD, 0, sizeof(TKD));
    // calculate TDK scores for NUM_OF_BUCKETS * TOPK movies
    for(int i = 0; i < at; ++i) {
        int p = ((result[i] << 16) >> 16);
        for(int j = 0; j < at; ++j) {
            if(i == j) continue; // dont need to compare with itself
            int q = ((result[j] << 16) >> 16);
            bool all_greater_or_equal = true;
            bool at_least_one_greater = false;
            for(int k = 1; k < MAX_USER_ID; ++k) {
                all_greater_or_equal &= (ratings[p][k] >= ratings[q][k]);
                at_least_one_greater |= (ratings[p][k] > ratings[q][k]);
            }
            if(all_greater_or_equal && at_least_one_greater) {
                TKD[i] += 1;
            }
        }
    }

    float weight_factor[at];
    // calculate weights for NUM_OF_BUCKETS * TOPK movies
    printf("no of unique users: %d\n", unique_user);
    for(int i = 0; i < at; ++i) {
        int movie_id = ((result[i] << 16) >> 16);
        int review_count = 0;
        for(int user_id = 0; user_id < MAX_USER_ID; ++user_id) {
            review_count += (ratings[movie_id][user_id] != -1);
        }
        printf("Review count for movie_id: %d is: %d\n", movie_id, review_count);
        weight_factor[i] = (float) review_count / (float) unique_user;
    }

    float weighted_tkd_score[at];
    for(int i = 0; i < at; ++i) {
        int movie_id = ((result[i] << 16) >> 16);
//        printf("id: %d, tkd_score: %d, weight_factor: %f\n", movie_id, TKD[i], weight_factor[i]);
        float weighted_score = (float) TKD[i] * weight_factor[i];
        weighted_tkd_score[i] = ldexp(weighted_score, 16);
//        weighted_tkd_score[i] += idexp(movie_id, 0);
        weighted_tkd_score[i] += (float) movie_id;
        printf("weighted_tkd_score: %f\n", weighted_tkd_score[i]);
    }

    // Problem below

    qsort(weighted_tkd_score, at, sizeof(float), compareWeightedScore);
    for(int i = 0; i < TOPK; ++i) {
        printf("weighted tkd score: %f\n", ldexp(weighted_tkd_score[i], -16));
    }


//    qsort(result, NUM_OF_BUCKETS * TOPK, sizeof(unsigned int), compareScore);
//    printf("Top-K results are: \n");
//    for(int i = 0; i < TOPK; ++i) {
//        printf("score: %d, id: %d\n", (result[i] >> 16), ((result[i] << 16) >> 16));
//    }
    return 0;
}



Writing dominating_query.c


In [4]:
!gcc dominating_query.c -o dominating_query
!./dominating_query

Failed to open the file.
